In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from fbprophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# 1. 데이터 불러오기 및 전처리
data = pd.read_csv("your_data_file.csv")
data["datetime"] = pd.to_datetime(data["datetime"])
data = data.sort_values(by="datetime")

In [ ]:
# 2. ARIMA 모델 훈련 및 예측
ts_data = data["visitors"]
ts_data.index = pd.DatetimeIndex(data["datetime"])

arima_model = SARIMAX(ts_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 24)).fit()
arima_forecast = arima_model.get_forecast(steps=24*7)


# 2. ARIMA 모델 훈련 및 예측
ts_data = data["visitors"]
ts_data.index = pd.DatetimeIndex(data["datetime"])

arima_model = SARIMAX(ts_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 24), exog=data["event"]).fit()
arima_forecast = arima_model.get_forecast(steps=24*7, exog=data["event"].tail(24*7))

In [ ]:
# 3. PROPHET 모델 훈련 및 예측
prophet_data = data[["datetime", "visitors"]]
prophet_data.columns = ["ds", "y"]

prophet_model = Prophet()
prophet_model.fit(prophet_data)

future = prophet_model.make_future_dataframe(periods=24*7, freq="H")
prophet_forecast = prophet_model.predict(future)

# 3. PROPHET 모델 훈련 및 예측
prophet_data = data[["datetime", "visitors", "event"]]
prophet_data.columns = ["ds", "y", "event"]

holidays = pd.DataFrame({
    "holiday": "event",
    "ds": prophet_data[prophet_data["event"] == 1]["ds"],
    "lower_window": 0,
    "upper_window": 1
})

prophet_model = Prophet(holidays=holidays)
prophet_model.fit(prophet_data.drop(columns=["event"]))

future = prophet_model.make_future_dataframe(periods=24*7, freq="H")
prophet_forecast = prophet_model.predict(future)

In [ ]:
# 4. 성능 비교 (MAE, RMSE)
actual = data["visitors"].tail(24*7)

arima_predicted = arima_forecast.predicted_mean[-(24*7):]
prophet_predicted = prophet_forecast["yhat"].tail(24*7)

mae_arima = mean_absolute_error(actual, arima_predicted)
mae_prophet = mean_absolute_error(actual, prophet_predicted)

rmse_arima = np.sqrt(mean_squared_error(actual, arima_predicted))
rmse_prophet = np.sqrt(mean_squared_error(actual, prophet_predicted))

print(f"ARIMA - MAE: {mae_arima}  RMSE: {rmse_arima}")
print(f"PROPHET - MAE: {mae_prophet} RMSE: {rmse_prophet}")

In [ ]:
# 지역별로 데이터를 분리
locations = data["location"].unique()

# 지역별 모델 및 예측 결과를 저장할 빈 딕셔너리
arima_results = {}
prophet_results = {}

# 각 지역별로 ARIMA 및 Prophet 모델 훈련 및 예측 수행
for location in locations:
    data_location = data[data["location"] == location]
    
    # ARIMA 모델
    ts_data_location = data_location["visitors"]
    ts_data_location.index = pd.DatetimeIndex(data_location["datetime"])
    arima_model_location = SARIMAX(ts_data_location, order=(1, 1, 1), seasonal_order=(1, 1, 1, 24)).fit()
    arima_forecast_location = arima_model_location.get_forecast(steps=24*7)
    arima_results[location] = arima_forecast_location.predicted_mean
    
    # Prophet 모델
    prophet_data_location = data_location[["datetime", "visitors"]]
    prophet_data_location.columns = ["ds", "y"]
    prophet_model_location = Prophet()
    prophet_model_location.fit(prophet_data_location)
    future_location = prophet_model_location.make_future_dataframe(periods=24*7, freq="H")
    prophet_forecast_location = prophet_model_location.predict(future_location)
    prophet_results[location] = prophet_forecast_location["yhat"].tail(24*7)
